|||
|---|---|
|Title|nwpackageversions|
|Author|numbworks|
|Version|1.1.0|
||Please check [docs/docs-nwpackageversions.md](../docs/docs-nwpackageversions.md) before proceeding.|

#### Demo

In [3]:
from typing import Optional, cast
from nwpackageversions import RequirementChecker, RequirementSummary

requirements_fp : str = r"/workspaces/nwpackageversions/data/requirements_demo.txt"
requirement_checker : RequirementChecker = RequirementChecker()
requirement_summary : Optional[RequirementSummary] = requirement_checker.try_check(file_path = requirements_fp)


The status checking operation has started!
The list of local packages will be loaded from the following 'file_path': '/workspaces/nwpackageversions/data/requirements_demo.txt'.
The 'waiting_time' between each fetching request will be: '5' seconds.
'3' local packages has been found and successfully loaded.
'0' unparsed lines.
Now starting to evaluate the status of each local package...
The status evaluation operation has been successfully completed.
{ 'description': 'The current version ('2.32.3') of 'requests' matches with the most recent release ('2.32.3', '2024-05-29').' }
{ 'description': 'The current version ('4.10.0') of 'beautifulsoup4' doesn't match with the most recent release ('4.13.0b2', '2024-03-20').' }
{ 'description': 'The current version ('3.4.3') of 'asyncio' matches with the most recent release ('3.4.3', '2015-03-10').' }
Now starting the creation of a status summary...
The status summary has been successfully created.
{ 'total_packages': '3', 'matching': '2', 'matchin

In [4]:
requirement_checker.log_requirement_summary(requirement_summary = cast(RequirementSummary, requirement_summary))


{ 'total_packages': '3', 'matching': '2', 'matching_prc': '66.67%', 'mismatching': '1', 'mismatching_prc': '33.33%' }
{ 'description': 'The current version ('2.32.3') of 'requests' matches with the most recent release ('2.32.3', '2024-05-29').' }
{ 'description': 'The current version ('4.10.0') of 'beautifulsoup4' doesn't match with the most recent release ('4.13.0b2', '2024-03-20').' }
{ 'description': 'The current version ('3.4.3') of 'asyncio' matches with the most recent release ('3.4.3', '2015-03-10').' }
